In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyParticleAngular
from msmrd2.integrators import overdampedLangevin as odLangevin
import msmrd2.tools.quaternions as quats
import msmrd2.tools.particleTools as particleTools
import msmrd2.tools.analysis as analysisTools
import timeit

In [ ]:
# Main parameters
numparticles = 2
boxsize = 6 #5 
D = 1.0
Drot = 1.0
numTrajectories = 1000

## Get bound2unbound FPT data of bechmark and MSMRD from files

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 10000
filename1 = '../../data/dimer/first_passage_times/patchyDimerFPTs_2bound_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
fptsA = []
fptsB = []
with open(filename1) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        if state == 'A':
            fptsA.append(time)
        elif state == 'B':
            fptsB.append(time)

In [ ]:
# Load FPT data obtained with MSMRD integrator
lagtime = 600
numTrajs = 10000
filename = '../../data/dimer/first_passage_times/MSMRDpatchyDimerFPTs_2bound_trajs' + str(numTrajs) + '_lagt' + str(lagtime) +  \
'_boxsize' + str(boxsize) + '.xyz'
MSMRDfptsA = []
MSMRDfptsB = []
with open(filename) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        if state == 'A':
            MSMRDfptsA.append(time)
        elif state == 'B':
            MSMRDfptsB.append(time)

## Plots of FPT data from benchmark and MSMRD simulations

In [ ]:
# Color definitions
c1 = '#ff3300' # for benchmark
c2 = '#0099ff' # for MSM/RD
# Alpha transparency definition
a1 = 0.2
a2 = 0.7

In [ ]:
hbins = range(0, 600,15)
plt.hist(fptsA, bins = hbins, alpha=0.5, edgecolor='None', label ='A');
plt.hist(fptsB, bins = hbins, alpha=0.5, edgecolor='None', label ='B');
plt.legend()

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark
numBootsrapSamples = 2000
mfptA, fptAstd = analysisTools.bootstrappingMFPTs(fptsA, numBootsrapSamples)
mfptB, fptBstd = analysisTools.bootstrappingMFPTs(fptsB, numBootsrapSamples)
print("Raw MFPTs (A and B): ", np.array(fptsA).mean(), np.array(fptsB).mean() )
print("Bootstrapping mean and std (A): ", mfptA, fptAstd)
print("Bootstrapping mean and std (B): ", mfptB, fptBstd)

In [ ]:
hbins = range(0, 600,15)
plt.hist(MSMRDfptsA, bins = hbins, alpha=0.5, label ='MSMRD-A');
plt.hist(MSMRDfptsB, bins = hbins, alpha=0.5, label ='MSMRD-B');
plt.legend()

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of MSMRD simulation
numBootsrapSamples = 2000
MSMRDmfptA, MSMRDfptAstd = analysisTools.bootstrappingMFPTs(MSMRDfptsA, numBootsrapSamples)
MSMRDmfptB, MSMRDfptBstd = analysisTools.bootstrappingMFPTs(MSMRDfptsB, numBootsrapSamples)
print("Raw MSMRD MFPTs (A and B): ",np.array(MSMRDfptsA).mean(), np.array(MSMRDfptsB).mean())
print("MSMRD bootstrapping mean and std (A): ", MSMRDmfptA, MSMRDfptAstd)
print("MSMRD bootstrapping mean and std (B): ", MSMRDmfptB, MSMRDfptBstd)

In [ ]:
# Normalizing weights
weightsA = np.ones_like(fptsA)/float(len(fptsA))
weightsAMSMRD = np.ones_like(MSMRDfptsA)/float(len(MSMRDfptsA))
# Plot histograms
hbins = range(0, 600,15)
plt.hist(fptsA, bins = hbins, alpha=a1, color=c1, weights = weightsA );
plt.hist(fptsA, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step', 
         facecolor = 'None', weights = weightsA, label ='A');
plt.hist(MSMRDfptsA, bins = hbins, alpha=a1, color=c2, weights = weightsAMSMRD);
plt.hist(MSMRDfptsA, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step', 
         weights = weightsAMSMRD, label ='MSMRD-A');
plt.xlim([0,500])
plt.legend()

In [ ]:
# Normalizing weights
weightsB = np.ones_like(fptsB)/float(len(fptsB))
weightsBMSMRD = np.ones_like(MSMRDfptsB)/float(len(MSMRDfptsB))
# Plot histograms
hbins = range(0, 600,15)
plt.hist(fptsB, bins = hbins, alpha=a1, color=c1, weights = weightsB);
plt.hist(fptsB, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsB, label ='B',);
plt.hist(MSMRDfptsB, bins = hbins, alpha=a1, color=c2, weights = weightsBMSMRD);
plt.hist(MSMRDfptsB, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsBMSMRD, label ='MSMRD-B',);
plt.xlim([0,500])
plt.legend()

In [ ]:
print(len(fptsB), len(MSMRDfptsB))
print(len(fptsA), len(MSMRDfptsA))
ratioFPT = float(len(fptsA))/float(len(fptsB))
ratioMSMRDFPT = float(len(MSMRDfptsA))/float(len(MSMRDfptsB))
print("Ratios:", ratioFPT, ratioMSMRDFPT )

## Get unbound2bound FPT data of bechmark and MSMRD from files

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 10000
filenameA = '../../data/dimer/first_passage_times/patchyDimerFPTs_A2unbound_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
filenameB = '../../data/dimer/first_passage_times/patchyDimerFPTs_B2unbound_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
fptsA2unbound = []
fptsB2unbound = []
with open(filenameA) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fptsA2unbound.append(time)
with open(filenameB) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fptsB2unbound.append(time)

In [ ]:
# Load FPT data obtained with MSMRD integrator
lagtime = 600
numTrajs = 10000
filenameA = '../../data/dimer/first_passage_times/MSMRDpatchyDimerFPTs_A2unbound_trajs' + str(numTrajs) + '_lagt' + str(lagtime) +  \
'_boxsize' + str(boxsize) + '.xyz'
filenameB = '../../data/dimer/first_passage_times/MSMRDpatchyDimerFPTs_B2unbound_trajs' + str(numTrajs) + '_lagt' + str(lagtime) +  \
'_boxsize' + str(boxsize) + '.xyz'
MSMRDfptsA2unbound = []
MSMRDfptsB2unbound = []
with open(filenameA) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfptsA2unbound.append(time)
with open(filenameB) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfptsB2unbound.append(time)

## Plots of FPT data from benchmark and MSMRD simulations

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark
numBootsrapSamples = 200
mfptA2unbound, fptA2unboundstd = analysisTools.bootstrappingMFPTs(fptsA2unbound, numBootsrapSamples)
mfptB2unbound, fptB2unboundstd = analysisTools.bootstrappingMFPTs(fptsB2unbound, numBootsrapSamples)
print("Raw MFPTs (A and B): ", np.array(fptsA2unbound).mean(), np.array(fptsB2unbound).mean() )
print("Bootstrapping mean and std (A): ", mfptA2unbound, fptA2unboundstd)
print("Bootstrapping mean and std (B): ", mfptB2unbound, fptB2unboundstd)
MSMRDmfptA2unbound, MSMRDfptA2unboundstd = analysisTools.bootstrappingMFPTs(MSMRDfptsA2unbound, numBootsrapSamples)
MSMRDmfptB2unbound, MSMRDfptB2unboundstd = analysisTools.bootstrappingMFPTs(MSMRDfptsB2unbound, numBootsrapSamples)
print("Raw MSMRD MFPTs (A and B): ",np.array(MSMRDfptsA2unbound).mean(), np.array(MSMRDfptsB2unbound).mean())
print("MSMRD bootstrapping mean and std (A): ", MSMRDmfptA2unbound, MSMRDfptA2unboundstd)
print("MSMRD bootstrapping mean and std (B): ", MSMRDmfptB2unbound, MSMRDfptB2unboundstd)

In [ ]:
# Plotting histogram for A to unbound FPTs
# Normalizing weights
weightsA = np.ones_like(fptsA2unbound)/float(len(fptsA2unbound))
weightsAMSMRD = np.ones_like(MSMRDfptsA2unbound)/float(len(MSMRDfptsA2unbound))
# Plot histograms
hbins = np.arange(0, 5, 0.1)
plt.hist(fptsA2unbound, bins = hbins, alpha=a1, color=c1, weights = weightsA);
plt.hist(fptsA2unbound, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsA, label ='A');
plt.hist(MSMRDfptsA2unbound, bins = hbins, alpha=a1, color=c2, weights = weightsAMSMRD);
plt.hist(MSMRDfptsA2unbound, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsAMSMRD, label ='MSMRD-A');
plt.xlim([0,3])
plt.legend()

In [ ]:
# Plotting histogram for B to unbound FPTs
# Normalizing weights
weightsB = np.ones_like(fptsB2unbound)/float(len(fptsB2unbound))
weightsBMSMRD = np.ones_like(MSMRDfptsB2unbound)/float(len(MSMRDfptsB2unbound))
# Plot histograms
hbins = np.arange(0, 5, 0.1)
plt.hist(fptsB2unbound, bins = hbins, alpha=a1, color=c1, weights = weightsB);
plt.hist(fptsB2unbound, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsB, label ='B');
plt.hist(MSMRDfptsB2unbound, bins = hbins, alpha=a1, color=c2, weights = weightsBMSMRD);
plt.hist(MSMRDfptsB2unbound, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsBMSMRD, label ='MSMRD-B');
plt.xlim([0,3])
plt.legend()